TODO:
* add plotting functionality
* add global package importing function
* Either use the guys web scraping code or trey again with
sec-edgar-downloader api
* maybe even try to implement the rest of the project on the data you already
 have, before moving on to downloading an extra data


In [5]:
# TEST CODE
import sys

import wrds
import pandas as pd
from numpy import iterable

In [6]:
# TEST CODE
conn = wrds.Connection(wrds_username='denisfench')

Loading library list...
Done


In [158]:
# TEST CODE
### Get S&P500 Index Membership from CRSP

sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2000'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])


### Add Other Company Identifiers from CRSP.MSENAMES
### - You don't need this step if only PERMNO is required
### - This step aims to add TICKER, SHRCD, EXCHCD and etc.

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt,
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

print("The length of the full SP500 list is: ", len(sp500_full))

The length of the full SP500 list is:  985474


In [159]:
# TEST CODE
# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]


### Add Other Company Identifiers from CRSP.MSENAMES

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt,
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]


### Add Compustat Identifiers

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno,
                  linktype, linkprim, linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

# Rearrange columns for final output

sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', 'linktype', \
                                  'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip',\
                     'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]


### Add CIKs and Link with SEC Index Files using CIK

names = conn.raw_sql(""" select gvkey, cik, sic, naics, gind, gsubind from comp.names """)

# Merge sp500 constituents table with names table
sp500 = pd.merge(sp500ccm, names, on='gvkey',  how='left')
sp500.head(20)
print(len(sp500))

133640


In [7]:
import wrds
import pandas as pd

# this function retrieves all sp500 records from the WRDS database
# the records are returned in Pandas dataframe format
def get_sp500_records():
    # establish connection to the WRDS database
    conn = wrds.Connection(wrds_username='denisfench')

    ### Get S&P500 Index Membership from CRSP
    sp500 = conn.raw_sql("""
                            select a.*, b.date, b.ret
                            from crsp.msp500list as a,
                            crsp.msf as b
                            where a.permno=b.permno
                            and b.date >= a.start and b.date<= a.ending
                            and b.date>='01/01/2000'
                            order by date;
                            """, date_cols=['start', 'ending', 'date'])

    ### Add Other Company Identifiers from CRSP.MSENAMES, such as TICKER, SHRCD, EXCHCD
    mse = conn.raw_sql("""
                            select comnam, ncusip, namedt, nameendt,
                            permno, shrcd, exchcd, hsiccd, ticker
                            from crsp.msenames
                            """, date_cols=['namedt', 'nameendt'])

    # if nameendt is missing then set to today date
    mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

    # Merge with SP500 data
    sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

    # Impose the date range restrictions
    sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                                & (sp500_full.date<=sp500_full.nameendt)]

    ### Add Other Company Identifiers from CRSP.MSENAMES
    mse = conn.raw_sql("""
                            select comnam, ncusip, namedt, nameendt,
                            permno, shrcd, exchcd, hsiccd, ticker
                            from crsp.msenames
                            """, date_cols=['namedt', 'nameendt'])

    # if nameendt is missing then set to today date
    mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

    # Merge with SP500 data
    sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

    # Impose the date range restrictions
    sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                                & (sp500_full.date<=sp500_full.nameendt)]

    ### Add Compustat Identifiers
    ccm=conn.raw_sql("""
                      select gvkey, liid as iid, lpermno as permno,
                      linktype, linkprim, linkdt, linkenddt
                      from crsp.ccmxpf_linktable
                      where substr(linktype,1,1)='L'
                      and (linkprim ='C' or linkprim='P')
                      """, date_cols=['linkdt', 'linkenddt'])

    # if linkenddt is missing then set to today date
    ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

    # Merge the CCM data with S&P500 data
    # First just link by matching PERMNO
    sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

    # Then set link date bounds
    sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                            &(sp500ccm['date']<=sp500ccm['linkenddt'])]

    # Rearrange columns for final output
    sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', 'linktype', \
                                      'linkprim', 'linkdt', 'linkenddt'])
    sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip',\
                         'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                         'gvkey', 'iid', 'start', 'ending', 'ret']]

    ### Add CIKs and Link with SEC Index Files using CIK
    names = conn.raw_sql(""" select gvkey, cik, sic, naics, gind, gsubind from comp.names """)

    # Merge sp500 constituents table with names table
    sp500 = pd.merge(sp500ccm, names, on='gvkey',  how='left')
    return sp500

In [8]:
# TEST CODE
# Getting a two-year sample of records for 2020 and 2021

sp500_sample = sp500.loc['01/01/2020' <= sp500.date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

sp500_sample = sp500_sample.loc[sp500.date <= '01/01/2022'][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

NameError: name 'sp500' is not defined

In [35]:
# TEST CODE
sp500_sample

,date,permno,comnam,ncusip,gvkey,iid,cik,ticker,sic,naics
120131,2020-01-31,34746.0,FIFTH THIRD BANCORP,31677310,004640,01,0000035527,FITB,6020,522110
120132,2020-01-31,75591.0,IDEX CORP,45167R10,015267,01,0000832101,IEX,3561,333914
120133,2020-01-31,14541.0,CHEVRON CORP NEW,16676410,002991,01,0000093410,CVX,2911,324110
120134,2020-01-31,14593.0,APPLE INC,03783310,001690,01,0000320193,AAPL,3663,334220
120135,2020-01-31,75341.0,DUKE REALTY CORP,26441150,013510,01,0000783280,DRE,6798,531120
...,...,...,...,...,...,...,...,...,...,...
132138,2021-12-31,93096.0,DOLLAR GENERAL CORP NEW,25667710,004016,02,0000029534,DG,5331,452319
132139,2021-12-31,19502.0,WALGREENS BOOTS ALLIANCE INC,93142710,011264,01,0001618921,WBA,5912,446110
132140,2021-12-31,69796.0,CONSTELLATION BRANDS INC,21036P10,002710,02,0000016918,STZ,2082,312120
132141,2021-12-31,62092.0,THERMO FISHER SCIENTIFIC INC,88355610,010530,01,0000097745,TMO,3826,334516


In [9]:
# this function retrieves a subsample of the sp500 records corresponding to
# the given dates (YYYY-MM-DD)
# EX: get_sp500_records_by_date(sp500, '2020-01-01', '2021-12-31')

def get_sp500_records_by_date(sp500_record, start_date, end_date):
    new_sp500_sample = sp500_record.loc[start_date <= sp500_record
        .date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

    new_sp500_sample = new_sp500_sample.loc[sp500_record.date <=
                                      end_date][['date',
                                                                        'permno',
                                                          'comnam',
                                                   'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

    return new_sp500_sample

In [85]:
# increment the current date by 1 month
def get_next_month(date, DATE_FORMAT="%Y-%m-%d", datetime_format=True):
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    datetime_date = datetime.strptime(date, DATE_FORMAT)
    new_date = datetime_date + relativedelta(months=1)
    if datetime_format:
        return new_date
    else:
        return new_date.strftime(DATE_FORMAT)


def get_next_day(date):
    from datetime import datetime
    from dateutil.relativedelta import relativedelta
    DATE_FORMAT = "%Y-%m-%d"
    datetime_date = datetime.strptime(date, DATE_FORMAT)
    new_date = datetime_date + relativedelta(days=1)
    return new_date.strftime(DATE_FORMAT)

In [158]:
'''
# EXPERIMENTING WITH A DIFFERENT EDGAR API PACKAGE BECAUSE OF THE 500 ERROR
# THAT THE PREVIOUS LIBRARY WAS THROWING

def get_filing_by_month(tickers, date, dir_name):
    from secedgar import FilingType, CompanyFilings
    from datetime import datetime
    num_filings = 0
    DATE_FORMAT="%Y-%m-%d"

    print("\n THE TICKERS ARE \n" )
    print(tickers)

    filings_10q = CompanyFilings(cik_lookup=tickers, filing_type=FilingType
                             .FILING_NT10Q, user_agent="df1911",
                                 start_date=datetime.strptime(date, DATE_FORMAT),
                                 end_date=get_next_month(date, DATE_FORMAT))

    filings_10k = CompanyFilings(cik_lookup=tickers, filing_type=FilingType
                                 .FILING_10K, user_agent="df1911",
                                 start_date=datetime.strptime(date,
                                                              DATE_FORMAT),
                                 end_date=get_next_month(date, DATE_FORMAT))


    if filings_10q.count != 0:
        filings_10q.save(dir_name, dir_pattern="cik_{cik}/{type}",
           file_pattern="{accession_number}")

    if filings_10k.count != 0:
        filings_10k.save(dir_name, dir_pattern="cik_{cik}/{type}", file_pattern="{accession_number}")

    num_filings += filings_10q.count() + filings_10k.count()

    return num_filings
'''

'\n# EXPERIMENTING WITH A DIFFERENT EDGAR API PACKAGE BECAUSE OF THE 500 ERROR\n# THAT THE PREVIOUS LIBRARY WAS THROWING\n\ndef get_filing_by_month(tickers, date, dir_name):\n    from secedgar import FilingType, CompanyFilings\n    from datetime import datetime\n    num_filings = 0\n    DATE_FORMAT="%Y-%m-%d"\n\n    print("\n THE TICKERS ARE \n" )\n    print(tickers)\n\n    filings_10q = CompanyFilings(cik_lookup=tickers, filing_type=FilingType\n                             .FILING_NT10Q, user_agent="df1911",\n                                 start_date=datetime.strptime(date, DATE_FORMAT),\n                                 end_date=get_next_month(date, DATE_FORMAT))\n\n    filings_10k = CompanyFilings(cik_lookup=tickers, filing_type=FilingType\n                                 .FILING_10K, user_agent="df1911",\n                                 start_date=datetime.strptime(date,\n                                                              DATE_FORMAT),\n                              

In [11]:
# COMMENTED OUT CODE WHILE EXPERIMENTING WITH A DIFFERENT EDGAR LOADER PACKAGE
from sec_edgar_downloader import Downloader

def get_filing_by_month(ticker, date, dir_name):
    num_filings = 0
    dl = Downloader(dir_name)
    num_filings += dl.get("10-Q", ticker, after=date, before=get_next_month
    (date))
    num_filings += dl.get("10-K", ticker, after=date, before=get_next_month
    (date))
    return num_filings
#
def get_filing_by_day(ticker, date, dir_name):
    num_filings = 0
    dl = Downloader(dir_name)
    num_filings += dl.get("10-Q", ticker, after=date, before=get_next_day
    (date))
    num_filings += dl.get("10-K", ticker, after=date, before=get_next_day
    (date))
    return num_filings

In [142]:
import time

# this function retrieves all S&P 500 10-Q or 10-K filings for a given month,
# if such a filing exists for a given company and writes them to a given directory
# the date should be passed in the following format: YYYY-MM-DD
# TODO: THIS FUNCTION IS NOT ABLE TO MATCH ANY OF THE DATES
def get_all_sp_filings_by_month(sp500_collection, date, dir_name):
    import calendar
    from datetime import datetime
    sp500_tickers = []
    num_filings = 0
    DATE_FORMAT = "%Y-%m-%d"
    datetime_date = datetime.strptime(date, DATE_FORMAT)
    current_date = date
    month_end_date = date[:8] + str(calendar.monthrange(datetime_date.year,
                                                datetime_date
                                      .month)[1])
    month_end_date = datetime.strptime(month_end_date, DATE_FORMAT)

    while datetime.strptime(current_date, DATE_FORMAT) <= month_end_date:
        sp500_companies = sp500_collection.loc[sp500_collection.date ==
                                               current_date][['date',
                                                                        'permno',
                                                          'comnam',
                                                   'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]


        sp500_tickers.extend(sp500_companies['ticker'].tolist())
        current_date = get_next_day(current_date)

    num_filings += get_filing_by_month(sp500_tickers,
                                       current_date,
                                       dir_name)

    return num_filings

# this function retrieves all S&P 500 10-Q or 10-K filings for a given day,
# if such a filing exists for a given company and writes them to a given directory
# the date should be passed in the following format: YYYY-MM-DD
def get_all_sp_filings_by_day(sp500_collection, date, dir_name):
    num_filings = 0
    sp500_companies = sp500_collection.loc[sp500_collection.date == date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]
    sp500_tickers = sp500_companies['ticker']
    print("get_all_sp_filings_by_day() retrieved  " + str(len(sp500_tickers)))

    for ticker in sp500_tickers:
        num_filings += get_filing_by_day(ticker, date, dir_name)

    return num_filings

In [14]:

# this function retrieves all S&P 500 10-Q and 10-K filings for a given period
def get_sp500_filings(sp500_collection, start_date, end_date, dir_name):
    from datetime import datetime
    print("Retrieving all the 10-Q and 10-K filings for a specified time "
          "starting from " + start_date + " until " + end_date)
    DATE_FORMAT = "%Y-%m-%d"
    current_date = start_date
    num_filings = 0
    while datetime.strptime(current_date, DATE_FORMAT) <= datetime.strptime(end_date, DATE_FORMAT):
        num_filings += get_all_sp_filings_by_month(sp500_collection, current_date,
                                                   dir_name)
        current_date = get_next_month(current_date)
        print("Retrieved " + str(num_filings) + " for " + str(current_date))
    return num_filings

In [ ]:
# TEST CODE

DATA_FOLDER = "data"

filings = get_all_sp_filings_by_month("2020-01-31", DATA_FOLDER)
filings

KeyboardInterrupt: 

In [ ]:
# TEST CODE
apple_filing = get_filing_by_day()

In [10]:
# TEST CODE
# parsing the records
from bs4 import BeautifulSoup

In [11]:
# TEST CODE
with open("data/sec-edgar-filings/AAPL/10-Q/0000320193-20-000010/filing-details.html") as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [15]:
from bs4 import BeautifulSoup
import os

# this function retrieves all tickers for all companies from a given directory
def get_all_filing_tickers(dir_name):
    files = os.listdir(dir_name + "/sec-edgar-filings")
    return files

In [234]:
# TEST CODE
print(get_all_filing_tickers("data"))

['VZ', 'AMZN', 'CNP', 'RCL', 'CAT', 'AAPL', 'KHC', 'AGN', 'ANET', 'CAH', 'FBHS', 'PFE', 'REG', 'APTV', 'AAL', 'CDW', 'MAR', 'VRSN', 'KMI', 'SPGI', 'ALLE', 'SYK', 'PEP', 'FRT', 'SNPS', 'PLD', 'MMM', 'EMN', 'AMT', 'ADI', 'MAA', 'DRE', 'FTV', 'MGM', 'VLO', 'EMR', 'SYY', 'GILD', 'SNA', 'MS', 'ALXN', 'CVX', 'NCLH', 'UHS', 'MO', 'TXN', 'UNM', 'BK', 'BLL', 'TTWO', 'HCA', 'CDNS', 'HII', 'CTSH', 'MCHP', 'VTR', 'HIG', 'ETFC', 'IPG', 'DISH', 'UNP', 'AMAT', 'LLY', 'NTAP', 'WAB', 'REGN', 'RSG', 'IR', 'IQV', 'LMT', 'CI', 'PPL', 'ANSS', 'EL', 'PSX', 'JNJ', 'QCOM', 'LNT', 'BAC', 'IT', 'SIVB', 'DHI', 'IRM', 'DGX', 'EQR', 'ED', 'DHR', 'HSIC', 'ALB', 'NLOK', 'WM', 'COG', 'RHI', 'TMO', 'ALL', 'BSX', 'FAST', 'ABBV', 'AFL', 'RMD', 'TWTR', 'EFX', 'LYB', 'PNR', 'KEY', 'ATVI', 'CHRW', 'COF', 'PRGO', 'VMC', 'HSY', 'T', 'CHTR', 'TAP', 'MCK', 'AIG', 'EBAY', 'F', 'NTRS', 'O', 'CFG', 'STE', 'PYPL', 'IBM', 'AMD', 'BWA', 'MHK', 'PG', 'INCY', 'XOM', 'USB', 'SO', 'LRCX', 'FOXA', 'DTE', 'BRK', 'ROP', 'AME', 'RJF', 'KMB'

In [16]:
from datetime import datetime

def parse_date(filed_as_of_date_line):
    DATE_FORMAT = "%Y%m%d"
    date = ""
    for ch in filed_as_of_date_line:
        if ch.isdigit():
            date += ch
    return datetime.strptime(date, DATE_FORMAT).date()

In [74]:
# TEST CODE

parse_date("FILED AS OF DATE:		20200129")

datetime.date(2020, 1, 29)

In [17]:
from bs4 import BeautifulSoup

FILING_DETAILS = "filing-details.html"
FULL_SUBMISSION = "full-submission.txt"
FILED_AS_OF_DATE_IDX = 7

# this function will search for a filing in a given directory based on the
# given ticker
# it will return a dictionary with a ticker and the date of filing as the key
# and filing text as the value
# currently the function doesn't indicate whether the filing is a 10-Q or 10-K
def get_filings_from_ticker(ticker, dir_name, sec_filings):
    for root, dirs, files in os.walk(dir_name + "/sec-edgar-filings/" + ticker):
        for f in files:
            if FILING_DETAILS in f:
                with open(root + "/" + f) as fp:
                    soup = BeautifulSoup(fp, "html.parser")
                    line_idx = 0
                    with open(root + "/" + FULL_SUBMISSION) as fs_fp:
                        while line_idx < FILED_AS_OF_DATE_IDX:
                            line_idx += 1
                            fs_fp.readline()
                        date = parse_date(fs_fp.readline())
                    sec_filings[(ticker, date)] = soup.get_text()
    return sec_filings

In [77]:
# TEST CODE
filings = {}
filings = get_filings_from_ticker("AAL", "data", filings)
print(filings)

{('AAL', datetime.date(2020, 2, 19)): "\n\n\nDocument\n\n\n1.61.61.61.61.61.61.61.60.6250.6250.6250.6250.6250.6250.6250.625P1YP1YP1YP1Yfalse--12-31FY20190000006201000000451541000000410000000.10.10.10.10.10.10.10.10.10.10.10.0110.011175000000010001750000000100010001000460610870100042820250610000.07310.08390.02990.030.050.07310.080.08390.02990.050.030.0550.03450.04050.03450.04050.300.050.150.100.450.10.050.550.800.250.400.100.0500.0200.15000.200.450.100.150.020.300.050.150.100.450.10.200.550.800.250.400.10.0500.0200.15000.200.450.100.150.02P25YP25Y0.03540.03720.03800.03740.03540.03720.03800.03741000000000010000000000P30YP10YP30YP10YP20YP5YP5YP3YP30YP10YP30YP10YP20YP5YP5YP3Y24000000002400000000216686117091401682202339531271660615734090566\n\n0000006201\n\n\n2019-01-01\n2019-12-31\n\n\n\n0000006201\n\nsrt:SubsidiariesMember\n\n\n\n2019-01-01\n2019-12-31\n\n\n\n0000006201\n\n\n2020-02-14\n\n\n\n0000006201\n\n\n2019-06-30\n\n\n\n0000006201\n\nsrt:SubsidiariesMember\n\n\n\n2020-02-14\n\n\n\n0

In [7]:
# TEST CODE
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# nltk.download()
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dennisfenchenko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
import ssl

# this function sets up SSL context for downloading packages from nltk library
def create_ssl_context_for_nltk():
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

In [19]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# a function that tokenizes 10-K or 10-Q corpus that
def tokenize_filing(filing_corpus):
    filing_tokenizer = RegexpTokenizer(r'\w+')
    return filing_tokenizer.tokenize(filing_corpus)

In [20]:
# a function to remove the stop words from the filings corpus
def filter_stopwords(tokenized_filing_corpus):
    stop_words = set(stopwords.words('english'))
    filing_corpus_filtered = [word for word in tokenized_filing_corpus if not \
        word.lower() in stop_words]
    return filing_corpus_filtered

def filter_numbers(filing_corpus):
    return [token for token in filing_corpus if not (token.isdigit()
                                         or token[0] == '-' and token[1:].isdigit())]

In [22]:
# TEST CODE
tokenized_aal_filing = tokenize_filing(filings.get('AAL'))
filtered_corpus = filter_stopwords(tokenized_aal_filing)
filtered_corpus = filter_numbers(filtered_corpus)
print("The size before the filtering " + str(len(tokenized_aal_filing)))
print("The size after the filtering " + str(len(filtered_corpus)))

The size before the filtering 118317
The size after the filtering 67698


In [109]:
# TEST CODE
# get CRSP stock performance for each given company in a given quarter
# we will use the Value-Weighted Return (including distributions) for that
conn.list_tables(library='crsp')

['acti',
 'asia',
 'asib',
 'asic',
 'asio',
 'asix',
 'bmdebt',
 'bmheader',
 'bmpaymts',
 'bmquotes',
 'bmyield',
 'bndprt06',
 'bndprt12',
 'bxcalind',
 'bxdlyind',
 'bxmthind',
 'bxquotes',
 'bxyield',
 'cap',
 'ccm_lookup',
 'ccm_qvards',
 'ccmxpf_linktable',
 'ccmxpf_lnkhist',
 'ccmxpf_lnkrng',
 'ccmxpf_lnkused',
 'comphead',
 'comphist',
 'compmaster',
 'contact_info',
 'crsp_cik_map',
 'crsp_daily_data',
 'crsp_header',
 'crsp_monthly_data',
 'crsp_names',
 'crsp_portno_map',
 'crsp_ziman_daily_index',
 'crsp_ziman_monthly_index',
 'cs20yr',
 'cs5yr',
 'cs90d',
 'cst_hist',
 'daily_nav',
 'daily_nav_ret',
 'daily_returns',
 'dividends',
 'dport1',
 'dport2',
 'dport3',
 'dport4',
 'dport5',
 'dport6',
 'dport7',
 'dport8',
 'dport9',
 'dsbc',
 'dsbo',
 'dse',
 'dse62',
 'dse62delist',
 'dse62dist',
 'dse62exchdates',
 'dse62names',
 'dse62nasdin',
 'dse62shares',
 'dseall',
 'dseall62',
 'dsedelist',
 'dsedist',
 'dseexchdates',
 'dsenames',
 'dsenasdin',
 'dseshares',
 'dsf',


In [115]:
# TEST CODE

company = conn.get_table(library='crsp', table='msp500p', obs=5)

In [116]:
# TEST CODE
company

,caldt,vwretd,vwretx,ewretd,ewretx,totval,totcnt,usdval,usdcnt,spindx,sprtrn
0,1925-12-31,NaN,NaN,NaN,NaN,15236829.5,89.0,NaN,NaN,12.46,NaN
1,1926-01-30,-0.001783,-0.003980,0.006457,0.003250,15277664.0,89.0,15236829.5,79.0,12.74,0.022472
2,1926-02-27,-0.033296,-0.037876,-0.039979,-0.042451,14712894.9,89.0,15277664.0,81.0,12.18,-0.043956
3,1926-03-31,-0.057708,-0.062007,-0.067915,-0.073275,14012079.2,89.0,14712894.9,81.0,11.46,-0.059113
4,1926-04-30,0.038522,0.034856,0.031441,0.027121,14500482.2,89.0,14012079.2,82.0,11.72,0.022688


In [12]:
# TEST CODE
# Perform sentiment analysis on filing documents
import pysentiment2 as ps

# get the Harvard general sentiment dictionary
hiv4 = ps.HIV4()

# get the Loughran and McDonald dictionary
lm = ps.LM()

In [25]:
# TEST CODE
# get the sentiment score for each filing document

# calculate the sentiment score using Harvard dictionary
hiv4_score = hiv4.get_score(filtered_corpus)
print("Harvard dictionary score: " + str(hiv4_score))

# calculate the sentiment score using Loughran dictionary
lm_score = lm.get_score(filtered_corpus)
print("Loughran dictionary score: " + str(lm_score))

Harvard dictionary score: {'Positive': 1979, 'Negative': 882, 'Polarity': 0.3834323661714672, 'Subjectivity': 0.04226121894232827}
Loughran dictionary score: {'Positive': 226, 'Negative': 366, 'Polarity': -0.23648648608701608, 'Subjectivity': 0.0087447191939386}


In [21]:
# Perform sentiment analysis on filing documents
import pysentiment2 as ps

# this function calculates the sentiment score of the text using a dictionary
# (Loughran or Harvard) passed in as a parameter
def get_sentiment_score(text, dictionary):
    # get the Harvard general sentiment dictionary
    hiv4 = ps.HIV4()
    # get the Loughran and McDonald dictionary
    lm = ps.LM()

    if dictionary == "Harvard":
        return hiv4.get_sentiment_score(text)

    if dictionary == "Loughran":
        return lm.get_sentiment_score(text)

    raise ValueError("The dictionary could not be found.")

In [130]:
# TEST CODE
# load CRSP data for calculating Excess return

CRSP_data_dir = "CRSP-data/"
crsp_df = pd.read_csv(CRSP_data_dir + "crsp.csv", parse_dates=['date'])
print(crsp_df.head())

/var/folders/4l/kfc4gh5d1jn6zkrmv51vvbyw0000gn/T/ipykernel_2735/2536905571.py:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_df = pd.read_csv(CRSP_data_dir + "crsp.csv", parse_dates=['date'])


   PERMNO       date        RET    vwretd
0   10026 2019-01-02  -0.024829  0.001796
1   10026 2019-01-03   0.014326 -0.021043
2   10026 2019-01-04   0.012725  0.033410
3   10026 2019-01-07   0.003935  0.009202
4   10026 2019-01-08   0.022626  0.010269


In [131]:
# TEST CODE
sp500_permno_nums = sp500_sample[['permno', 'ticker']].copy()
print(sp500_permno_nums.head())

         permno ticker
120131  34746.0   FITB
120132  75591.0    IEX
120133  14541.0    CVX
120134  14593.0   AAPL
120135  75341.0    DRE


In [132]:
# TEST CODE
print(type(crsp_df['PERMNO'][0]))
print(type(sp500_permno_nums['permno']))
print(sp500_permno_nums.iloc[0])

<class 'numpy.int64'>
<class 'pandas.core.series.Series'>
permno    34746.0
ticker       FITB
Name: 120131, dtype: object


In [22]:
# this function retrieves Value-Weighted Return (including distributions) and
# Holding Period Return from the CRSP database
def get_crsp_data():
    CRSP_DATA_DIR = "CRSP-data/"
    crsp_df = pd.read_csv(CRSP_DATA_DIR + "crsp.csv", parse_dates=['date'])
    return crsp_df

In [23]:
# this function calculates excess return using Value-Weighted Return (including distributions) and
# Holding Period Return from the CRSP database
def get_excess_return(crsp_dataframe, permno, date):
    excess_return_df = crsp_dataframe[crsp_dataframe['PERMNO'] == permno]
    excess_return_df =  excess_return_df[crsp_dataframe['date'] == date]
    excess_return = float(excess_return_df['vwretd']) - \
                    float(excess_return_df['RET'])
    return excess_return

In [135]:
# TEST CODE
excess_return_test = get_excess_return(crsp_df, 10026, "2019-01-02")
print(excess_return_test)

0.026625


/var/folders/4l/kfc4gh5d1jn6zkrmv51vvbyw0000gn/T/ipykernel_2735/1184843951.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  excess_return_df =  excess_return_df[crsp_dataframe['date'] == date]


In [24]:
DICTIONARY_DIR = "dictionaries/"

# this function constructs a sentiment dictionary based on the Loughran
# McDonald and Harvard dictionaries
# return: {Term: [str], Sentiment: [int], Term_Count: [int]}
# where Term count is the number of times the term occurs within the corpus
def construct_sentiment_dict(base_dict="Loughran"):
    if base_dict == "Loughran":
        df = pd.read_csv(DICTIONARY_DIR +
                         "Loughran-McDonald_MasterDictionary_1993-2021.csv",
                         header=0, usecols=['Word', 'Negative', 'Positive'])
    else:
        df = pd.read_csv(DICTIONARY_DIR + "Harvard.csv", header=0, usecols=['Word', 'Negative', 'Positive'])

    df = df[(df['Negative'] != 0) | (df['Positive'] != 0)]
    return df

In [25]:
lr_dict = construct_sentiment_dict()
lr_dict.head()

,Word,Negative,Positive
9,ABANDON,2009,0
10,ABANDONED,2009,0
11,ABANDONING,2009,0
12,ABANDONMENT,2009,0
13,ABANDONMENTS,2009,0


In [26]:
# This function constructs a term dictionary based on a given document: tokenized filing list
# return: {Term: [str], Term_Count: [int]}
# where Term count is the number of times the term occurs within the document
def construct_term_dict(document):
    doc_dict = {}
    for token in document:
        if token in doc_dict:
            doc_dict[token] += 1
        else:
            doc_dict[token] = 1
    return doc_dict

In [27]:
# This function calculates a document negativity score based on the
# proportion of negative words in the document
# the negativity of the term is determined based on a given dictionary
# (Loughran or Harvard)
def compute_negativity_score_based_on_proportion(text,
                                                 sentiment_dict="Loughran"):
    num_negative = 0
    sentiment_score = get_sentiment_score(text, sentiment_dict)
    num_negative = sentiment_score['Negative']
    return num_negative / len(text)

In [ ]:
# TEST CODE

corpus_dict = construct_sentiment_dict()

In [ ]:
# This function calculates a document negativity score based on the
# TFIDF score for the document
# TFIDF:
# TF (Term Frequency) (t, d) = (# Occurrences of the word in a document / # Total words in a document )
# IDF (Inverse Document Frequency) (t, D) = log ( (N (number of documents in a collection) / (# Documents containing the term t))
# TFIDF (t, d, D) = TF (t, d) * IDF (t, D)
# the function takes document dictionary, containing terms and their
# frequencies for a given document; the sentiment dictionary, either Loughran
# or Harvard, containing token sentiment rating as well as the document term
# count for the entire corpus; the number of documents in the collection and
# the number of tokens within a given document
def get_tfidf_negativity_score(doc_dict, sentiment_dict, doc_term_freq,
                               collection_size):
    pass

In [196]:
from sklearn.feature_extraction.text import TfidfVectorizer

# This function calculates TFIDF score for the filings using SciKit Learn
def calc_tfidf_score(sentiment_dict, corpus):
    vectorizer = TfidfVectorizer(vocabulary=sentiment_dict.keys(),
                                 stop_words="english")
    vectors = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()
    dense_list = dense.tolist()
    tfidf_df = pd.DataFrame(dense_list, columns=feature_names)
    return tfidf_df

In [195]:
def load_filings_corpus(corpus_root_dir):
    sec_filings_corpus = []
    FILING_DETAILS = "filing-details.html"
    for root, dirs, files in os.walk(corpus_root_dir + "/sec-edgar-filings/"):
        for f in files:
            if FILING_DETAILS in f:
                with open(root + "/" + f) as file_ptr:
                    bs = BeautifulSoup(file_ptr, "html.parser")
                    sec_filings_corpus.append(bs.get_text())
    return sec_filings_corpus

In [191]:
test_filings = load_filings_corpus("data")

In [194]:
print(len(test_filings))
print(test_filings)
sys.getsizeof(test_filings)

5
['\n\n\nDocument\n\n\nfalse--12-31FY2019000073271276500000073300000002.3352.3852.4350.10.162500000006250000000429143364642914336464100000002250000001500000000.003300.00450.00420.00370.007700.038750.050.008750.01250.0250.008750.0150.018750.03560.01420.080.08380.08750.07880.060.05130.05510.0680.08950.01380.01380.02650.0430.08950.051500.06550.0260.05050.01750.03850.0210.026250.02650.0350.0420.0590.08750.013750.01750.030.0350.040160.0410.032P3Y750000000P13YP25YP8YP2Y011000000002210000000023200000000100000000200000020000000190000002650000003000000011000000210000001440000002840000002190000006580000006580000000.10.125000000025000000000P20YP45YP50YP50YP20YP5YP7YP7YP3YP3YP20YP45YP50YP50YP20YP5YP7YP7YP3YP3YP5YP1YP1Y15940026715560552700300000000200000000P2YP2Y54000000\n\n0000732712\n\n\n2019-01-01\n2019-12-31\n\n\n\n0000732712\n\nvz:A2.875NotesDue2038Member\n\n\n\n2019-01-01\n2019-12-31\n\n\n\n0000732712\n\nvz:A1.875NotesDue2030Member\n\n\n\n2019-01-01\n2019-12-31\n\n\n\n0000732712\n\nvz:A2.625

120

<h1>SENTIMENT ANALYSIS OF 10-Qs and 10-Ks for S&P 500 companies, years 2020,
2021<h1>

In [29]:
# getting all S&P 500 records from the WRDS database

sp500_all = get_sp500_records()

Loading library list...
Done


In [102]:
# retrieving all S&P 500 records for the two-year period (from '2020-01-01'
# to '2021-12-31')
sp500_sample = get_sp500_records_by_date(sp500_all, '2020-01-01', '2021-12-31')

In [157]:
sp500_sample.head()

,date,permno,comnam,ncusip,gvkey,iid,cik,ticker,sic,naics
120131,2020-01-31,34746.0,FIFTH THIRD BANCORP,31677310,004640,01,0000035527,FITB,6020,522110
120132,2020-01-31,75591.0,IDEX CORP,45167R10,015267,01,0000832101,IEX,3561,333914
120133,2020-01-31,14541.0,CHEVRON CORP NEW,16676410,002991,01,0000093410,CVX,2911,324110
120134,2020-01-31,14593.0,APPLE INC,03783310,001690,01,0000320193,AAPL,3663,334220
120135,2020-01-31,75341.0,DUKE REALTY CORP,26441150,013510,01,0000783280,DRE,6798,531120


In [93]:
# experimenting with dates retrieval
# sp500_sub_sample = sp500.loc['01/01/2020' == sp500.date][['date',
#                                                                     'permno',
#                                                       'comnam',
#                                                'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

sp500_sub_sample = sp500_sample.loc['2020-01-31' == sp500_sample.date][['date',
                                                                    'permno',
                                                      'comnam',
                                               'ncusip', 'gvkey', 'iid', 'cik', 'ticker', 'sic', 'naics']]

list_of_tickers = sp500_sub_sample['ticker'].tolist()

In [113]:
print(str(type(list_of_tickers[0])))
print(list_of_tickers)

<class 'str'>
['FITB', 'IEX', 'CVX', 'AAPL', 'DRE', 'NLOK', 'PXD', 'SCHW', 'HLT', 'AMAT', 'VTR', 'ALLE', 'CCL', 'EOG', 'INFO', 'ANET', 'EA', 'TWTR', 'SLB', 'ABMD', 'TIF', 'CSCO', 'BKR', 'COG', 'SYF', 'HOLX', 'AMGN', 'CFG', 'KEYS', 'SYK', 'COTY', 'STT', 'XLNX', 'VIAC', 'NWSA', 'MRO', 'TFC', 'CDW', 'PVH', 'UNM', 'COP', 'AZO', 'REGN', 'ECL', 'VMC', 'C', 'IQV', 'KHC', 'MO', 'IDXX', 'PYPL', 'HOG', 'AES', 'VRTX', 'ZBRA', 'PEP', 'STZ', 'RJF', 'MYL', 'BIIB', 'MS', 'ODFL', 'ZTS', 'TXN', 'KIM', 'NCLH', 'ABBV', 'HPE', 'QCOM', 'PRGO', 'PEAK', 'GILD', 'EIX', 'AIG', 'ROP', 'SNPS', 'WDC', 'FTV', 'M', 'FANG', 'HD', 'MHK', 'PNR', 'LW', 'USB', 'BSX', 'NOW', 'FB', 'KSS', 'STE', 'T', 'PSX', 'FTI', 'HCN', 'DHI', 'VZ', 'HSY', 'COO', 'SBUX', 'AOS', 'TSN', 'KEY', 'KR', 'CPRI', 'D', 'APTV', 'DD', 'PSA', 'XYL', 'AGN', 'PGR', 'CVS', 'NSC', 'INTU', 'MCHP', 'FBHS', 'LB', 'CB', 'MPC', 'GIS', 'CI', 'ORLY', 'RCL', 'KSU', 'SPGI', 'WRB', 'FLIR', 'HII', 'GL', 'ALL', 'CSX', 'BWA', 'HCA', 'EQR', 'UHS', 'ATVI', 'IT', 'TMO'

In [143]:
start_date = '2020-01-01'
end_date = '2021-12-31'
sp_filings_dir_name = 'sp500_filings_sample'
num_filings_sample = get_sp500_filings(sp500_sample, start_date, end_date, sp_filings_dir_name)

Retrieving all the 10-Q and 10-K filings for a specified time starting from 2020-01-01 until 2021-12-31

 THE TICKERS ARE 

['FITB', 'IEX', 'CVX', 'AAPL', 'DRE', 'NLOK', 'PXD', 'SCHW', 'HLT', 'AMAT', 'VTR', 'ALLE', 'CCL', 'EOG', 'INFO', 'ANET', 'EA', 'TWTR', 'SLB', 'ABMD', 'TIF', 'CSCO', 'BKR', 'COG', 'SYF', 'HOLX', 'AMGN', 'CFG', 'KEYS', 'SYK', 'COTY', 'STT', 'XLNX', 'VIAC', 'NWSA', 'MRO', 'TFC', 'CDW', 'PVH', 'UNM', 'COP', 'AZO', 'REGN', 'ECL', 'VMC', 'C', 'IQV', 'KHC', 'MO', 'IDXX', 'PYPL', 'HOG', 'AES', 'VRTX', 'ZBRA', 'PEP', 'STZ', 'RJF', 'MYL', 'BIIB', 'MS', 'ODFL', 'ZTS', 'TXN', 'KIM', 'NCLH', 'ABBV', 'HPE', 'QCOM', 'PRGO', 'PEAK', 'GILD', 'EIX', 'AIG', 'ROP', 'SNPS', 'WDC', 'FTV', 'M', 'FANG', 'HD', 'MHK', 'PNR', 'LW', 'USB', 'BSX', 'NOW', 'FB', 'KSS', 'STE', 'T', 'PSX', 'FTI', 'HCN', 'DHI', 'VZ', 'HSY', 'COO', 'SBUX', 'AOS', 'TSN', 'KEY', 'KR', 'CPRI', 'D', 'APTV', 'DD', 'PSA', 'XYL', 'AGN', 'PGR', 'CVS', 'NSC', 'INTU', 'MCHP', 'FBHS', 'LB', 'CB', 'MPC', 'GIS', 'CI', 'ORLY', '

/Users/dennisfenchenko/NYU-Fall-2022/nlp-finance/10k-sentiment-analysis/venv/lib/python3.9/site-packages/secedgar/cik_lookup.py:144: UserWarning: Lookup 'DISCA' will be skipped.
                          Found multiple companies matching 'DISCA':
                          Discala Abraxas J
Wheat Sheaf Brands, Inc.SIC: 3089 - PLASTICS PRODUCTS, NEC
  warnings.warn(warning_message)
/Users/dennisfenchenko/NYU-Fall-2022/nlp-finance/10k-sentiment-analysis/venv/lib/python3.9/site-packages/secedgar/cik_lookup.py:144: UserWarning: Lookup 'CERN' will be skipped.
                          Found multiple companies matching 'CERN':
                          Cern Corp
Cern One Ltd
Cern Robert A
CERN YEHUDA
Cerna Ana Cecilia
CERNA JAMES J JR
CERNER CorpSIC: 7373 - SERVICES-COMPUTER INTEGRATED SYSTEMS DESIGN
CERNER DHT INCSIC: 7373 - SERVICES-COMPUTER INTEGRATED SYSTEMS DESIGN
CERNIC EDWARD J SR
CERNICALO SHIPPING LLC
CERNIGLIA DOMINIC
Cerniglia Kristina A.
CERNIGLIARO CARMEN
CERNIUM CORP
CERNIUM INC

EDGARQueryError: No results were found or the value submitted was not valid.

In [121]:
print(my_filings.count)

None


In [125]:

from datetime import date

filing = CompanyFilings(cik_lookup=["aapl", "msft", "fb"],
                        filing_type=FilingType.FILING_10K,
                        start_date=date(2015, 1, 1),
                        user_agent="df1911@nyu.edu")

In [126]:
print(filing.)

None


In [132]:

from datetime import date
import secedgar as sec
quarterly = sec.QuarterlyFilings(year=2021,
                                 quarter=2,
                                 user_agent="df1911@nyu.edu")

In [133]:
quarterly.save("secedgar_test")

  0%|          | 0/300040 [00:00<?, ?it/s]


ClientConnectorCertificateError: Cannot connect to host www.sec.gov:443 ssl:True [SSLCertVerificationError: (1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')]

In [153]:
from edgar import Company, XBRL, XBRLElement

company = Company("Oracle Corp", "0001341439")
results = company.get_data_files_from_10K("EX-101.INS", isxml=True)
xbrl = XBRL(results[0])
XBRLElement(xbrl.relevant_children_parsed[15]).to_dict()

IndexError: list index out of range

In [154]:
from sec_api import QueryApi

# get your API key at https://sec-api.io
query_api = QueryApi(api_key='8f477658b7d17069a35d1153c3b0460f816f6d1f37e7873182ff015065b73ea5')

In [155]:
query = {
    "query": {
        "query_string": {
            "query": "(formType:\"10-Q\" OR formType:\"10-K\") AND filedAt:{2020-01-01 TO 2021-12-31}"
        }
    },
    "from": "0",
    "size": "20",
    "sort": [{ "filedAt": { "order": "desc" } }]
}

query_result = query_api.get_filings(query)

In [156]:
len(query_result)

3

In [ ]:
# PERHAPS TRY USING sec_edgar_downloader with CIK numbers instead of tickers.
# This is the only API that really worked.

In [160]:
from sec_edgar_downloader import Downloader


dl = Downloader("/sec_edgar_downloader_test")

# Get the five most recent 8-K filings for Apple
dl.get("8-K", "0000320193", amount=5)

OSError: [Errno 30] Read-only file system: '/sec_edgar_downloader_test'